## GA-output

Nb to test the output files and params of  the GASS

In [ ]:
using PyCall
using DataFrames

rootdir = "/home/stephane/Science/ALMA/ArrayConfig/GASS"
#rootdir = "/home/stephane/alma/ArrayConfig/GASS"

push!(LOAD_PATH,"$rootdir/master/src")
using GASS
using Printf

import PyPlot
import Random
import JLD

np= pyimport("numpy")


## directory
datadir = "$rootdir/master/data"
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"

cd(wdir)

In [ ]:
macro main(inpfile)
    cfg = read_cfg(inpfile)
    
    res= gass_optimization(cfg)
    
    return(res)
end

res= @main("../master/data/GA_Parameters_O-3.txt.julia")

In [ ]:
println(res[1].age)
println(res[1].fitness)
println(res[5].score)

JLD.save("test.jld", "gass", res)

In [ ]:
a = JLD.load("test.jld")
a["gass"][5].subarr[5,2]

In [ ]:
### save array in CASA format..
###
function _save_CASAarr(fileprefix::String, cfg::cfg, bestsub)
    println("Saving results")
    println(cfg.arr)
    println(cfg.sub)
    #println(p.subarr)
    #println(p.score)
    #println(p.fitness)
    
    headerCasa= "# observatory=ALMA\n# coordsys=LOC (local tangent plane)\n# x y z diam pad#\n"
    
    for i in 1:length(bestsub)
        arrStr= string(headerCasa)
        subname= cfg.sub.Subarray_Name[i]
        # println(cfg.arr.X[bestsub[i]])
        println(length(bestsub[i]))
        for j in 1:length(bestsub[i])
            arrStr *= @sprintf("%f %f %f %2.1f %s \n",cfg.arr.X[bestsub[i][j]], cfg.arr.Y[bestsub[i][j]] , 
                cfg.arr.Z[bestsub[i][j]] , cfg.arr.diam[bestsub[i][j]] , cfg.arr.name[bestsub[i][j]])
        end
        println(arrStr)
        filename= fileprefix*subname*".cfg"
        println(fileprefix)
        println(subname)
        println(filename)
        f= open(filename, "w")
        print(f,arrStr)
        close(f)
    end
end

## sort population wrt. score
##
function _sort_population(p::population)
    println("Sorting..")
    s= sortperm(p.score, rev=true)
    println(s)
    psorted= population(p.age, p.subarr[s,:], p.fitness[s,:], p.score[s], p.param[s,:])
    println(psorted.fitness)
    return(psorted)
end

In [ ]:
inpfile= "../master/data/GA_Inputs_O-3.txt.julia"
a= read_cfg(inpfile , verbose=true)
println(typeof(res[1]))


psort= _sort_population(res[5])
## psort.subarr[1,:] is the first best solution...
#println(psort.subarr[1,:])
_save_CASAarr("toto-",a, psort.subarr[1,:])


In [ ]:
## full run with JLD output ..
##
function _full_run(fileprefix::String, cfg::cfg, jld= false)
    println(fileprefix)
    res= gass_optimization(cfg)

    lastPopulation= cfg.ga.Number_Iterations
    psort= _sort_population(res[lastPopulation])
    _save_CASAarr(fileprefix, cfg, psort.subarr[1,:])
    
    if jld
       JLD.save(fileprefix*".jld", "evol", res)
       println("## $fileprefix.jld written..")
    end
    
    return(res)
end

In [ ]:
inpfile= "../master/data/GA_Parameters_O-3.txt.julia"
cfg2 = read_cfg(inpfile)
t= @time _full_run("toto_" , cfg2 , true)